# 実験1 パーサ結合子の基本

## 1-1 ライブラリ（cumps.js）の読み込み（最初に必ず必要）

In [ ]:
require("./cumps");
function print(s) { console.log(s); }

## 1-2 数（10進）を切り足すパーサnumの定義

ライブラリ関数patに正規表現を与えるとそれを切り出すパーサを返す．

In [ ]:
var num = pat("[0-9]+")

入力文字列"123abc"を与えてnumを呼び出すと結果としてオブジェクトが返る．
オブジェクトには切り出した文字列"123"と残りの文字列"abc"が含まれる．

In [ ]:
num("123abc") // "123abc"を他の文字列に変えて試してみよ

## 1-3 英字（小文字）の並びを切り足すパーサalphaの定義

In [ ]:
var alpha = pat("[a-z]+");

In [ ]:
alpha("abc123")

[課題1] 整数だけでなく小数も切り出せるパーサfnumを定義せよ．適切なテスト例を考えてテストを行え．

In [ ]:
var fnum = pat("ここにどのような正規表現を与えればよいか？");

テスト例（これだけでは十分ではない．他の必要なテスト例は自分で考えよ．）

In [ ]:
fnum("123abc")

In [ ]:
fnum("123.456abc")

In [ ]:
fnum("123.abc")

In [ ]:
fnum("123.456.789abc")

## 1-4 パーサの直列接続

複数のパーサを順につなげれば流れ作業による処理が可能になる．それにはライブラリ関数seqにつなぎたいパーサを順に与える．以下はコンマで区切られた3つの数を切り出すパーサcsv3を定義する例．ライブラリ関数wに文字列を与えると、その文字列を切り出すパーサを返す．

In [ ]:
var comma = w(",");

In [ ]:
var csv3 = seq(num,comma,num,comma,num);

コンマを区切りにして切り出した3つの数と2つのコンマが順に配列に格納されていることが分かる．4つ目の数以降は切り出されないでやり残しの文字列に含まれている．

In [ ]:
csv3("12,3,456,78,9")

数とコンマの間にある空白は自動的に無視される．

In [ ]:
csv3("12, 3    ,   456    ,78,  9")

In [ ]:
解析結果に区切り記号のコンマを含めたくないときは、w(",")の代わりに単に","を用いる．

In [ ]:
var csv3 = seq(num,",",num,",",num);

In [ ]:
今度は配列に3つの数が順に格納されている．

In [ ]:
csv3("12,3,456,78,9")

## 1-5 パーサの反復接続

パーサを0回以上繰り返し用いて処理したいときはライブラリ関数moreThan0を用いる．以下は0個以上の数の並びを切り出すパーサnumsを定義する例．

In [ ]:
var nums = moreThan0(num);

In [ ]:
nums("")

In [ ]:
nums("12 3 456 7")

seqとmoreThan0を組み合わせればコンマで区切られた任意個の数の並びを切り出せる．これは「形式言語とオートマトン」の授業で触れた「カツサンド」パターンの一例になっている（数がパン、コンマがカツ）．

In [ ]:
var csv = seq(num, moreThan0(seq(",", num)));

今度は4つ目の数以降も正しく切り出されている．

In [ ]:
csv("12, 3,456,78, 9")

「カツサンド」パターンは実用上頻出するので、専用のライブラリ関数sepByが用意されている．sepByにパンとカツを与えるとカツサンドのパターンを切り出すパーサを返す．これを用いると上の例と同じことがずっと簡単に書ける．

In [ ]:
var csv = sepBy(num,","); 

In [ ]:
csv("12, 3,456,78, 9")

## 1-6 パーサの並列接続

複数のパーサを並列につなぐと後のものが先のもののバックアップ（下支え、予備）として働く．以下に定義するパーサgeq_or_gthは、まず不等号>=を切り出そうと試み、それが失敗したときには不等号>を切り出そうとする．

In [ ]:
var geq_or_gth = oneOf(w(">="), w(">"));

In [ ]:
geq_or_gth(">=0")

In [ ]:
geq_or_gth(">0")

後の処理が試されるのは、先の処理が失敗した場合に限られることに注意．以下のパーサgth_or_geqは決して>=を切り出すことがない．>=の切り出しが
成功するときには必ず>の切り出しも成功するからである．

In [ ]:
var gth_or_geq = oneOf(w(">"), w(">="));

In [ ]:
gth_or_geq(">=0")

## 1-7 オプション（省略可能な処理）の指定

処理の省略を許す場合はライブラリ関数optを用いる．以下はコンマ区切で区切られた数の並びを切り出すが、末尾に余分にコンマがあってもよい点が以前と異なる．

In [ ]:
var csv = seq(sepBy(num,","), opt(","))

In [ ]:
csv("12,3,45")

In [ ]:
csv("12,3,45,")

## 1-8 パーサの再帰的定義

パーサの定義は再帰的であってもよい．つまり左辺の変数が右辺の式の中に出現してもよい．ただしダミーの（仮の）関数定義を前に置く必要がある．そうでないと左辺の変数が参照する関数がセットされる前に右辺の変数が参照されるのでエラーになる．

例：正しく対応のついたかっこの並びは以下の文法で表せるので再帰的に定義するのが自然である．

（例）正しく対応のついたかっこの並び
$E \rightarrow \varepsilon \ \mid \ ( E ) \ E$
（あるいは右辺に正規表現を流用して
$E \rightarrow ( E )*$）

後者の文法にしたがうなら以下のような定義が可能．

In [ ]:
var pexp = function(s) { return pexp(s); }; // ダミーの関数を変数pexpにセットしておく
var pexp = moreThan0(seq(w("("),pexp,w(")")));

In [ ]:
pexp("((())())")

In [ ]:
pexp("())(")

In [ ]:
pexp(")()(")

前者の文法にしがたうなら以下のような定義も可能．

In [ ]:
var pexp2 = function(s) { return pexp2(s); }; // ダミーの関数を変数pexpにセットしておく
var pexp2 = opt(seq(w("("),pexp2,w(")"),pexp2));

In [ ]:
pexp2("((())())")

[課題2] pexpあるいはpexp2を拡張して，丸かっこだけでなくブラケット（[,]）とブレース（{,}）も許すパーサxpexpを定義せよ．適切なテスト例を考えテストを行え．

In [ ]:
var xpexp = function(s){ return xpexp(s); };
// 以下自分で
// var xpexp = ...

In [ ]:
xpexp("{[()]()}")

In [ ]:
xpexp("(]")